In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("..")
sys.path.append("../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import SLNet, SUnet, SL_AEnet, AEnet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [4]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
enc_nf = [64, 64, 64, 64]
dec_nf = [64, 64, 64, 64]

In [5]:
net1 = AEnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=10,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          10
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  51%|█████     | 466/916 [00:25<00:30, 14.97batch/s]


Validation round: 100%|█████████▉| 914/916 [00:49<00:00, 16.97batch/s]
                                                                      INFO: Validation Dice Loss: 0.13058229136447458
Validation round:   3%|▎         | 25/916 [00:02<02:58,  4.99batch/s]


Validation round:  54%|█████▍    | 499/916 [00:27<00:29, 14.17batch/s]


Validation round:  99%|█████████▉| 909/916 [00:50<00:00, 17.68batch/s]
                                                                      INFO: Validation Dice Loss: 0.10700247618146078
Validation round:   8%|▊         | 76/916 [00:04<00:56, 14.97batch/s]


Validation round:  67%|██████▋   | 610/916 [00:33<00:20, 15.02batch/s]


...

Validation round:  99%|█████████▉| 911/916 [00:50<00:00, 18.99batch/s]
                                                                      INFO: Validation Dice Loss: 0.06535759134681725
Validation round:  22%|██▏       | 197/916 [00:12<00:44, 16.17batch/s]


Validation round:  71%|███████   | 651/916 [00:37<00:19, 13.85batch/s]


Validation round: 100%|█████████▉| 914/916 [00:52<00:00, 18.42batch/s]
                                                                      INFO: Validation Dice Loss: 0.06547288091519775
Validation round:  16%|█▌        | 145/916 [00:07<00:42, 18.07batch/s]


Validation round:  65%|██████▌   | 596/916 [00:32<00:20, 15.40batch/s]


Validation round: 100%|█████████▉| 915/916 [00:50<00:00, 18.84batch/s]
                                                                      INFO: Validation Dice Loss: 0.06550250538344206
Epoch 10/10: 100%|██████████| 7329/7329 [05:13<00:00, 23.38img/s, loss (batch)=0.108] 
INFO: Checkpoint 10 saved !


In [ ]:
net2 = SL_AEnet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=10,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          10
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  49%|████▉     | 448/916 [00:21<00:26, 17.68batch/s]


Validation round:  94%|█████████▍| 864/916 [00:43<00:03, 16.85batch/s]


Validation round: 100%|██████████| 916/916 [00:46<00:00, 17.51batch/s]
                                                                      INFO: Validation Dice Loss: 0.20659269462441252
Validation round:  47%|████▋     | 432/916 [00:22<00:22, 21.40batch/s]


Validation round:  99%|█████████▊| 903/916 [00:46<00:00, 14.67batch/s]


Validation round: 100%|█████████▉| 913/916 [00:46<00:00, 16.02batch/s]
                                                                      INFO: Validation Dice Loss: 0.19548434323293673
Validation round:  52%|█████▏    | 473/916 [00:26<00:20, 21.20batch/s]


...

Validation round: 100%|█████████▉| 915/916 [00:46<00:00, 17.58batch/s]
                                                                      INFO: Validation Dice Loss: 0.12598074852248986
Validation round:   2%|▏         | 22/916 [00:01<02:56,  5.07batch/s]


Validation round:  59%|█████▊    | 536/916 [00:26<00:14, 26.46batch/s]


Validation round: 100%|█████████▉| 913/916 [00:40<00:00, 32.82batch/s]
                                                                      INFO: Validation Dice Loss: 0.10682747187702937
Validation round:  25%|██▌       | 231/916 [00:09<00:27, 24.52batch/s]


Validation round: 100%|█████████▉| 912/916 [00:37<00:00, 24.58batch/s]
                                                                      INFO: Validation Dice Loss: 0.10377879963789184
Validation round:  11%|█▏        | 105/916 [00:04<00:38, 21.13batch/s]


Validation round:  98%|█████████▊| 897/916 [00:36<00:00, 24.62batch/s]

In [ ]:
net3 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net3.to(device=device)
train_scores3, val_scores3, train_var_3, val_var_3 = train_net(net=net3,
                                                                epochs=10,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=1,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

In [ ]:
net4 = SLNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net4.to(device=device)
train_scores4, val_scores4, train_var_4, val_var_4 = train_net(net=net4,
                                                                epochs=10,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

In [ ]:
print("configuring combined plots")
domain = len(train_scores1)
x_values = [i+1 for i in range(domain)]
             
a3 = plt.subplot(1,2,1)
a3.set_ylim([0, 0.5])
plt.title("Simple AENet: 304,719 par")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores1, train_var_1)]
ziptraindown = [a - b for a, b in zip(train_scores1, train_var_1)]
zipvalup = [a + b for a, b in zip(val_scores1, val_var_1)]
zipvaldown = [a - b for a, b in zip(val_scores1, val_var_1)]

plt.plot(x_values, train_scores1, color="blue", label="train")
a3.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores1, color="orange", label="val")
a3.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a4 = plt.subplot(1,2,2)
a4.set_ylim([0, 0.5])
plt.title("Tied Simple AENet: 46,223 par")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores2, train_var_2)]
ziptraindown = [a - b for a, b in zip(train_scores2, train_var_2)]
zipvalup = [a + b for a, b in zip(val_scores2, val_var_2)]
zipvaldown = [a - b for a, b in zip(val_scores2, val_var_2)]

plt.plot(x_values, train_scores2, color="blue", label="train")
a4.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores2, color="orange", label="val")
a4.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

a2 = plt.subplot(1,2,2)
a2.set_ylim([0, 0.5])
plt.title("Simple Net: 452,175 par")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores3, train_var_3)]
ziptraindown = [a - b for a, b in zip(train_scores3, train_var_3)]
zipvalup = [a + b for a, b in zip(val_scores3, val_var_3)]
zipvaldown = [a - b for a, b in zip(val_scores3, val_var_3)]

plt.plot(x_values, train_scores3, color="blue", label="train")
a2.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores3, color="orange", label="val")
a2.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a1 = plt.subplot(1,2,1)
a1.set_ylim([0, 0.5])
plt.title("Simple TiedNet: 120,015 par")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores4, train_var_4)]
ziptraindown = [a - b for a, b in zip(train_scores4, train_var_4)]
zipvalup = [a + b for a, b in zip(val_scores4, val_var_4)]
zipvaldown = [a - b for a, b in zip(val_scores4, val_var_4)]

plt.plot(x_values, train_scores4, color="blue", label="train")
a1.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores4, color="orange", label="val")
a1.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

plt.close()